In [6]:
!pip install torcheval
!pip install pytorch-ignite
!pip install -r requirements.txt 

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 9.7 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=d1925f8092238056e78f14391ee6e9ce5a9636259161ec0dbf6d738e5dc8a55a
  Stored in directory: /Users/brunoneira/Library/Caches/pip/wheels/1f/be/48/13754633f1d08d1fbfc60d5e80ae1e5d7329500477685286cd
Successfully built antlr4-python3-runtime


In [8]:
# Perform Imports
import sys
sys.path.append(".")
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torcheval.metrics as metrics
import torchvision.transforms as T
import torchvision.datasets as datasets
import torchvision.transforms.functional as TF
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from t2i_model import T2IEncoderInput


# Setup

In [ ]:
# Some preliminary setup to load the dataset
batch_size  = 256
d_proj      = 512
d_hidden    = 32
config_path = "logs/vqgan_gumbel_f8/configs/model.yaml"
chkpt_path = "logs/vqgan_gumbel_f8/checkpoints/last.ckpt"

transform = T.Compose([
    T.PILToTensor(), # Convert all PIL objects to tensors
    T.Resize((d_hidden, d_hidden)),  # Resize all images to d_h x d_h
    T.ConvertImageDtype(torch.float),
])

# Define a custom collate_fn
def custom_collate_fn(batch):
    images, captions = zip(*batch)

    # Find the max height and width in the batch
    max_height = max(img.shape[1] for img in images)
    max_width  = max(img.shape[2] for img in images)

    # Pad images to the max dimensions
    padded_images = [
        F.pad(img, (0, max_width - img.shape[2], 0, max_height - img.shape[1]))
        for img in images
    ]

    # Stack images and return with captions
    return torch.stack(padded_images), captions

# Define helper functions
def normalize_output(x):
  x = torch.clamp(x, -1., 1.)
  x = (x + 1.)/2.
  return x

# load the dataset
dataset = datasets.CocoCaptions(root = 'TEST_ROOT_PATH',
                                annFile = 'TEST_ANN_FILE',
                                transform=transform)
# load the dataloader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn = custom_collate_fn)

# load the model checkpoint

'''
TODO: CHANGE THIS FOR THE DIFFRENT MODELS
'''
model = T2IEncoderInput(d_proj, d_hidden, config_path, chkpt_path, True)
model.load_state_dict(torch.load(PATH, weights_only=True))
model.eval() # set it to evaluation mode

# Getting FID for model checkpoint

In [ ]:
# create the FID object
'''
TODO: specify the device as part of the FID object
TODO: might want to update the number of features in the FID object
'''
FID = metrics.FrechetInceptionDistance()

# loop through the dataloader
# Convert caption tuple options into a list
for (img, txt) in tqdm(dataloader):
   txt = [t[0] for t in txt]

   # Pass it through our model and get the logits
   txt_to_img = model(x = txt)
   txt_to_img = torch.stack([normalize_output(t) for t in txt_to_img], dim = 0)

   # add original and generated images to the FID object
   FID.update(images=img, is_real=True)
   FID.update(images=txt_to_img, is_real=False)

# calculate the FID score
fid_score = FID.compute()

print("FID Score: ", fid_score)

# Get Inception Score

In [9]:
from collections import OrderedDict

import torch
from torch import nn, optim

from ignite.engine import *
from ignite.handlers import *
from ignite.metrics import *
from ignite.metrics.regression import *
from ignite.utils import *

/opt/anaconda3/lib/python3.12/site-packages/ignite/handlers/checkpoint.py:16: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


In [ ]:
IS = InceptionScore()

for (img, txt) in tqdm(dataloader):
    txt = [t[0] for t in txt]

   # Pass it through our model and get the logits
    txt_to_img = model(x = txt)
    txt_to_img = torch.stack([normalize_output(t) for t in txt_to_img], dim = 0)

    # add original and generated images to the FID object
    IS.update(output=txt_to_img)

# calculate the FID score
is_score = IS.compute()